##Teacher (VGG-16)

In [ ]:
# ==============================================================
# STEP 1 — Setup and Train/Load Teacher (VGG-16) for CIFAR-100
# ==============================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm

# --------------------------------------------------------------
# Configuration
# --------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
num_classes = 100
epochs = 10   # Fine-tuning only, not full training
save_path = "teacher_vgg16.pth"

# --------------------------------------------------------------
# Data Preparation
# --------------------------------------------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size,
                        shuffle=False, num_workers=2)

# --------------------------------------------------------------
# Define Teacher Model (Pretrained VGG-16)
# --------------------------------------------------------------
from torchvision.models import vgg16

def get_teacher_model(num_classes=100):
    model = vgg16(weights='IMAGENET1K_V1')  # Load pretrained weights
    model.classifier[6] = nn.Linear(4096, num_classes)  # Replace final layer
    return model

teacher = get_teacher_model(num_classes).to(device)
print(f"Loaded Teacher VGG-16 model on {device}")

# --------------------------------------------------------------
# Define Loss and Optimizer
# --------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(teacher.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# --------------------------------------------------------------
# Training Loop (Fine-tuning)
# --------------------------------------------------------------
def train_teacher(model, trainloader, testloader, epochs, save_path):
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Evaluation
        acc = evaluate(model, testloader)
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(trainloader):.4f} | Test Acc: {acc:.2f}%")

        # Save best model
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), save_path)

    print(f"✅ Training complete. Best Test Accuracy: {best_acc:.2f}%")
    print(f"Model saved to {save_path}")

# --------------------------------------------------------------
# Evaluation Function
# --------------------------------------------------------------
def evaluate(model, dataloader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# --------------------------------------------------------------
# Run Training (if checkpoint not found)
# --------------------------------------------------------------
if not os.path.exists(save_path):
    print("🚀 Fine-tuning teacher on CIFAR-100...")
    train_teacher(teacher, trainloader, testloader, epochs, save_path)
else:
    teacher.load_state_dict(torch.load(save_path))
    print("✅ Loaded existing pretrained teacher model.")


100%|██████████| 169M/169M [00:03<00:00, 49.2MB/s]


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:02<00:00, 200MB/s]


Loaded Teacher VGG-16 model on cuda
🚀 Fine-tuning teacher on CIFAR-100...


Epoch 1/10: 100%|██████████| 391/391 [00:36<00:00, 10.69it/s]


...

Epoch [8/10] Loss: 1.2257 | Test Acc: 61.86%


Epoch 9/10: 100%|██████████| 391/391 [00:37<00:00, 10.52it/s]


Epoch [9/10] Loss: 1.1675 | Test Acc: 62.95%


Epoch 10/10: 100%|██████████| 391/391 [00:37<00:00, 10.50it/s]


Epoch [10/10] Loss: 1.1143 | Test Acc: 62.68%
✅ Training complete. Best Test Accuracy: 62.95%
Model saved to teacher_vgg16.pth


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import shutil

# # Source path where PyTorch saved the model
# src_path = 'teacher_vgg16.pth'

# # Destination path in your Google Drive
# dst_path = '/content/drive/MyDrive/teacher_vgg16.pth'

# # Copy file to Drive
# shutil.copy(src_path, dst_path)
# print("✅ Model saved to Google Drive at:", dst_path)


✅ Model saved to Google Drive at: /content/drive/MyDrive/teacher_vgg16.pth


In [ ]:
# import torch
# import torchvision.models as models
# from torch import nn

# # Mount Drive if needed
# from google.colab import drive
# drive.mount('/content/drive')

# # Define the same model structure used during training
# model = models.vgg16(weights=None)

# # Modify the classifier to match CIFAR-100 (100 output classes)
# model.classifier[6] = nn.Linear(4096, 100)

# # Load your trained weights
# checkpoint_path = '/content/drive/MyDrive/teacher_vgg16.pth'
# model.load_state_dict(torch.load(checkpoint_path))

# # Move to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# model.eval()

# print("🚀 Model loaded successfully with 100-class output and ready for inference!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Model loaded successfully with 100-class output and ready for inference!


##Student Independent

In [ ]:
"""
Independent Student (SI) Training on CIFAR-100
----------------------------------------------
This trains a VGG-11 student model from scratch *without* any teacher guidance.
It serves as the baseline for Knowledge Distillation comparisons.
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import os

# ======================================================
# 1. Device Setup
# ======================================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# ======================================================
# 2. Dataset Preparation
# ======================================================
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),
                         (0.2673, 0.2564, 0.2762)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),
                         (0.2673, 0.2564, 0.2762)),
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

# ======================================================
# 3. Model Definition (VGG11)
# ======================================================
from torchvision.models import vgg11

def build_student(num_classes=100):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

student = build_student(num_classes=100).to(DEVICE)

# ======================================================
# 4. Loss, Optimizer, and Scheduler
# ======================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)

# ======================================================
# 5. Training & Evaluation Loops
# ======================================================
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100.0 * correct / total

best_acc = 0.0
num_epochs = 30
save_path = "student_vgg11_SI.pth"

# ======================================================
# 6. Training Loop
# ======================================================
for epoch in range(num_epochs):
    student.train()
    running_loss = 0.0
    pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in pbar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = student(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix(loss=running_loss / (len(pbar) + 1))

    scheduler.step()
    val_acc = evaluate(student, testloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(trainloader):.4f} - Val Acc: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(student.state_dict(), save_path)
        print(f"✅ New best model saved with accuracy: {best_acc:.2f}%")

print(f"\nTraining finished! Best validation accuracy: {best_acc:.2f}%")
print(f"Model saved to: {save_path}")


Using device: cuda


100%|██████████| 169M/169M [00:07<00:00, 24.0MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/30: 100%|██████████| 391/391 [00:30<00:00, 12.96it/s, loss=4.38]


Epoch 1/30 - Loss: 4.3905 - Val Acc: 5.24%
✅ New best model saved with accuracy: 5.24%


Epoch 2/30: 100%|██████████| 391/391 [00:29<00:00, 13.31it/s, loss=3.94]


Epoch 2/30 - Loss: 3.9548 - Val Acc: 10.19%
✅ New best model saved with accuracy: 10.19%


...

Epoch 28/30 - Loss: 1.1816 - Val Acc: 55.51%


Epoch 29/30: 100%|██████████| 391/391 [00:30<00:00, 13.01it/s, loss=1.13]


Epoch 29/30 - Loss: 1.1310 - Val Acc: 55.68%


Epoch 30/30: 100%|██████████| 391/391 [00:30<00:00, 12.86it/s, loss=1.08]


Epoch 30/30 - Loss: 1.0853 - Val Acc: 56.81%
✅ New best model saved with accuracy: 56.81%

Training finished! Best validation accuracy: 56.81%
Model saved to: student_vgg11_SI.pth


In [ ]:
# import shutil
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Local model file (after training)
# local_path = "student_vgg11_SI.pth"

# # Destination in Google Drive
# drive_path = "/content/drive/MyDrive/student_vgg11_SI.pth"

# # Copy file
# shutil.copy(local_path, drive_path)
# print(f"✅ Model saved to Google Drive at: {drive_path}")

Mounted at /content/drive
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_SI.pth


##LOGIT MATCHING

In [ ]:
# ==============================================================
# KNOWLEDGE DISTILLATION — LOGIT MATCHING (TUNED VERSION)
# ==============================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg11, vgg16
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
num_classes = 100
epochs = 30
teacher_ckpt = "teacher_vgg16.pth"
student_ckpt = "student_vgg11_LM_tuned.pth"

# --------------------------------------------------------------
# Data setup
# --------------------------------------------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# --------------------------------------------------------------
# Model definitions
# --------------------------------------------------------------
def get_teacher(num_classes=100):
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def get_student(num_classes=100):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

teacher = get_teacher(num_classes).to(device)
teacher.load_state_dict(torch.load(teacher_ckpt, map_location=device))
teacher.eval()

student = get_student(num_classes).to(device)

# --------------------------------------------------------------
# Evaluation
# --------------------------------------------------------------
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            _, preds = torch.max(out, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return 100 * correct / total

# --------------------------------------------------------------
# KD Loss
# --------------------------------------------------------------
class KDLoss(nn.Module):
    def __init__(self, T=4.0, alpha=0.5):
        super(KDLoss, self).__init__()
        self.T = T
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels):
        ce_loss = self.ce(student_logits, labels)
        soft_teacher = F.softmax(teacher_logits.detach() / self.T, dim=1)
        soft_student = F.log_softmax(student_logits / self.T, dim=1)
        kl_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (self.T ** 2)
        return self.alpha * ce_loss + (1 - self.alpha) * kl_loss

# --------------------------------------------------------------
# Training
# --------------------------------------------------------------
def train_kd(student, teacher, trainloader, testloader, epochs=30, T=4.0, alpha=0.5, lr=0.01):
    criterion = KDLoss(T=T, alpha=alpha)
    optimizer = optim.SGD(student.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    best_acc = 0.0

    for epoch in range(epochs):
        student.train()
        running_loss = 0.0
        for imgs, labels in tqdm(trainloader, desc=f"KD Epoch {epoch+1}/{epochs}"):
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.no_grad():
                teacher_logits = teacher(imgs)

            student_logits = student(imgs)
            loss = criterion(student_logits, teacher_logits, labels)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), 5.0)
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        acc = evaluate(student, testloader)
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(trainloader):.4f} | Acc: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            torch.save(student.state_dict(), student_ckpt)

    print(f"✅ KD Training complete. Best Acc: {best_acc:.2f}% | Saved at {student_ckpt}")

# --------------------------------------------------------------
# Run tuned KD training
# --------------------------------------------------------------
train_kd(student, teacher, trainloader, testloader,
         epochs=30, T=4.0, alpha=0.5, lr=0.01)


KD Epoch 1/30: 100%|██████████| 391/391 [00:41<00:00,  9.40it/s]


Epoch [1/30] | Loss: 4.9112 | Acc: 6.49%


KD Epoch 2/30: 100%|██████████| 391/391 [00:41<00:00,  9.43it/s]


Epoch [2/30] | Loss: 4.2187 | Acc: 12.25%


KD Epoch 3/30: 100%|██████████| 391/391 [00:41<00:00,  9.45it/s]


...

Epoch [28/30] | Loss: 0.9532 | Acc: 59.61%


KD Epoch 29/30: 100%|██████████| 391/391 [00:41<00:00,  9.43it/s]


Epoch [29/30] | Loss: 0.9458 | Acc: 59.55%


KD Epoch 30/30: 100%|██████████| 391/391 [00:41<00:00,  9.42it/s]


Epoch [30/30] | Loss: 0.9396 | Acc: 59.60%
✅ KD Training complete. Best Acc: 59.61% | Saved at student_vgg11_LM_tuned.pth


In [ ]:
# import shutil
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Local model file (after training)
# local_path = "student_vgg11_LM_tuned.pth"

# # Destination in Google Drive
# drive_path = "/content/drive/MyDrive/student_vgg11_LM_tuned.pth"

# # Copy file
# shutil.copy(local_path, drive_path)
# print(f"✅ Model saved to Google Drive at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_LM_tuned.pth


##Label Smoothing Regularization (LSR)

In [ ]:
# ==============================================================
# Label Smoothing Regularization (LSR) Implementation
# ==============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------------------------------------------------------------
# 1. Define Label Smoothing Loss
# --------------------------------------------------------------
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=100, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))


# --------------------------------------------------------------
# 2. CIFAR-100 Data Loaders
# --------------------------------------------------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)


# --------------------------------------------------------------
# 3. Define Student Model (VGG-11)
# --------------------------------------------------------------
def get_vgg11_student():
    model = models.vgg11_bn(weights=None, num_classes=100)
    return model.to(device)


# --------------------------------------------------------------
# 4. Evaluation Function
# --------------------------------------------------------------
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


# --------------------------------------------------------------
# 5. Training Function (Label Smoothing)
# --------------------------------------------------------------
def train_label_smoothing(model, trainloader, testloader, epochs=30, lr=0.01, smoothing=0.1):
    criterion = LabelSmoothingLoss(classes=100, smoothing=smoothing)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    best_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(trainloader, desc=f"LS Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        acc = evaluate(model, testloader)
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(trainloader):.4f} | Acc: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "student_vgg11_LS.pth")

    print(f"✅ Label Smoothing Training complete. Best Acc: {best_acc:.2f}% | Saved at student_vgg11_LS.pth")
    return best_acc


# --------------------------------------------------------------
# 6. Run Training
# --------------------------------------------------------------
student_lsr = get_vgg11_student()
acc_ls = train_label_smoothing(student_lsr, trainloader, testloader, epochs=30, lr=0.01, smoothing=0.1)


LS Epoch 1/30: 100%|██████████| 391/391 [00:30<00:00, 12.74it/s]


Epoch [1/30] | Loss: 4.2494 | Acc: 11.73%


LS Epoch 2/30: 100%|██████████| 391/391 [00:30<00:00, 12.69it/s]


Epoch [2/30] | Loss: 3.9034 | Acc: 15.19%


LS Epoch 3/30: 100%|██████████| 391/391 [00:30<00:00, 12.84it/s]


...

Epoch [28/30] | Loss: 1.6873 | Acc: 60.88%


LS Epoch 29/30: 100%|██████████| 391/391 [00:30<00:00, 12.88it/s]


Epoch [29/30] | Loss: 1.6666 | Acc: 60.88%


LS Epoch 30/30: 100%|██████████| 391/391 [00:30<00:00, 12.84it/s]


Epoch [30/30] | Loss: 1.6610 | Acc: 61.11%
✅ Label Smoothing Training complete. Best Acc: 61.11% | Saved at student_vgg11_LS.pth


In [ ]:
# import shutil
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Local model file (after training)
# local_path = "student_vgg11_LS.pth"

# # Destination in Google Drive
# drive_path = "/content/drive/MyDrive/student_vgg11_LS.pth"

# # Copy file
# shutil.copy(local_path, drive_path)
# print(f"✅ Model saved to Google Drive at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_LS.pth


##Decoupled Knowledge Distillation (DKD)

In [ ]:
"""
task3_dkd.py

Decoupled Knowledge Distillation (DKD) training script for:
- Teacher: pretrained/fine-tuned VGG-16 on CIFAR-100 (teacher_vgg16.pth)
- Student: VGG-11 trained with DKD
"""

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg11, vgg16
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

# ---------------------------
# Config
# ---------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_CLASSES = 100
NUM_EPOCHS = 30
TEACHER_CKPT = "teacher_vgg16.pth"
STUDENT_OUT = "student_vgg11_DKD.pth"
LR = 0.01
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9

# DKD hyperparameters
TEMPERATURE = 4.0           # T
LAMBDA_CE = 0.5             # weight for CE(hard labels) vs distillation
ALPHA_DKD = 0.5             # weight for TCKD (target)
BETA_DKD = 0.5              # weight for NCKD (non-target)
EPS = 1e-8

# ---------------------------
# Helpers: data loaders
# ---------------------------
def get_dataloaders(batch_size=BATCH_SIZE, num_workers=2):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761))
    ])
    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return trainloader, testloader

# ---------------------------
# Models
# ---------------------------
def build_teacher(num_classes=NUM_CLASSES):
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def build_student(num_classes=NUM_CLASSES):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

# ---------------------------
# Evaluation
# ---------------------------
def evaluate(model, dataloader, device=DEVICE):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model(imgs)
            preds = logits.argmax(dim=1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return 100.0 * correct / total

# ---------------------------
# DKD loss implementation
# ---------------------------
class DKDLoss(nn.Module):
    def __init__(self, T=TEMPERATURE, alpha=ALPHA_DKD, beta=BETA_DKD, eps=EPS):
        super().__init__()
        self.T = float(T)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.eps = float(eps)

    def forward(self, student_logits, teacher_logits, targets):
        """
        Inputs:
          - student_logits: (B, C)
          - teacher_logits: (B, C)
          - targets: (B,) long
        Returns: scalar DKD distillation loss
        """
        B, C = student_logits.shape
        T = self.T
        # teacher and student probabilities with temperature
        p_t = F.softmax(teacher_logits / T, dim=1)            # (B, C)
        p_s = F.softmax(student_logits / T, dim=1)            # (B, C)

        # target-class probabilities
        targets_col = targets.view(-1, 1)                     # (B,1)
        p_t_true = p_t.gather(1, targets_col).squeeze(1)      # (B,)
        p_s_true = p_s.gather(1, targets_col).squeeze(1)      # (B,)

        # === TCKD: target class knowledge ===
        # Weighted negative log-likelihood of student true prob, weighted by teacher true prob
        # tckd = - mean( p_t_true * log(p_s_true) )
        tckd = - torch.mean(p_t_true.detach() * torch.log(p_s_true + self.eps))

        # === NCKD: non-target knowledge ===
        # mask out true class
        mask = torch.ones_like(p_t, device=p_t.device)
        mask.scatter_(1, targets_col, 0.0)                    # zeros at true class positions

        # teacher and student non-target distributions
        p_t_nt = p_t * mask                                    # (B, C) with zero at target
        p_s_nt = p_s * mask

        # normalize so each row sums to 1 over non-target classes (avoid divide by zero)
        denom_t = p_t_nt.sum(dim=1, keepdim=True) + self.eps
        denom_s = p_s_nt.sum(dim=1, keepdim=True) + self.eps
        p_t_nt_norm = p_t_nt / denom_t                         # (B, C)
        p_s_nt_norm = p_s_nt / denom_s

        # compute KL divergence per sample between teacher non-target and student non-target
        # KL( p_t_nt_norm || p_s_nt_norm ) = sum p_t_nt_norm * (log p_t_nt_norm - log p_s_nt_norm)
        # sum across classes then mean over batch
        kl_nt = p_t_nt_norm * (torch.log(p_t_nt_norm + self.eps) - torch.log(p_s_nt_norm + self.eps))
        nckd = torch.mean(torch.sum(kl_nt, dim=1))

        # combine with weights. Note: scale NCKD by T^2 (as usual for scaled soft targets)
        loss = self.alpha * tckd + self.beta * (T * T) * nckd
        return loss

# ---------------------------
# Training routine
# ---------------------------
def train_dkd(teacher, student, trainloader, testloader,
              epochs=NUM_EPOCHS, lr=LR, lambda_ce=LAMBDA_CE, device=DEVICE):
    teacher.eval()
    student.train()
    dkd_criterion = DKDLoss(T=TEMPERATURE, alpha=ALPHA_DKD, beta=BETA_DKD, eps=EPS)
    ce = nn.CrossEntropyLoss()
    optimizer = optim.SGD(student.parameters(), lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    best_acc = 0.0
    for epoch in range(1, epochs+1):
        student.train()
        running_loss = 0.0
        iters = 0
        pbar = tqdm(trainloader, desc=f"DKD Epoch {epoch}/{epochs}")
        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.no_grad():
                t_logits = teacher(imgs)        # teacher frozen

            s_logits = student(imgs)
            loss_ce = ce(s_logits, labels)
            loss_dkd = dkd_criterion(s_logits, t_logits, labels)

            loss = lambda_ce * loss_ce + (1.0 - lambda_ce) * loss_dkd

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_norm=5.0)
            optimizer.step()

            running_loss += float(loss.item())
            iters += 1

        scheduler.step()
        avg_loss = running_loss / max(iters, 1)
        acc = evaluate(student, testloader, device)
        print(f"Epoch {epoch}/{epochs} - Loss: {avg_loss:.4f} - Val Acc: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            torch.save(student.state_dict(), STUDENT_OUT)
            print(f"  -> New best saved: {STUDENT_OUT} (acc {best_acc:.2f}%)")

    print(f"DKD training finished. Best Val Acc: {best_acc:.2f}%")

# ---------------------------
# Main
# ---------------------------
def main():
    trainloader, testloader = get_dataloaders(batch_size=BATCH_SIZE, num_workers=2)

    # load teacher checkpoint
    if not os.path.exists(TEACHER_CKPT):
        raise FileNotFoundError(f"Teacher checkpoint not found: {TEACHER_CKPT}. Please place it here.")
    teacher = build_teacher(NUM_CLASSES).to(DEVICE)
    teacher.load_state_dict(torch.load(TEACHER_CKPT, map_location=DEVICE))
    teacher.eval()
    print("Loaded teacher:", TEACHER_CKPT)

    # build student
    student = build_student(NUM_CLASSES).to(DEVICE)
    print("Student built, start DKD training...")

    train_dkd(teacher, student, trainloader, testloader,
              epochs=NUM_EPOCHS, lr=LR, lambda_ce=LAMBDA_CE, device=DEVICE)

    final_acc = evaluate(student, testloader, DEVICE)
    print("Final student accuracy (DKD):", final_acc)
    print("Saved best student to", STUDENT_OUT)

if __name__ == "__main__":
    main()

Loaded teacher: teacher_vgg16.pth
Student built, start DKD training...


DKD Epoch 1/30: 100%|██████████| 391/391 [00:41<00:00,  9.40it/s]


Epoch 1/30 - Loss: 3.1099 - Val Acc: 6.54%
  -> New best saved: student_vgg11_DKD.pth (acc 6.54%)


DKD Epoch 2/30: 100%|██████████| 391/391 [00:41<00:00,  9.47it/s]


Epoch 2/30 - Loss: 2.7394 - Val Acc: 11.74%
  -> New best saved: student_vgg11_DKD.pth (acc 11.74%)


...

Epoch 28/30 - Loss: 0.8959 - Val Acc: 57.28%


DKD Epoch 29/30: 100%|██████████| 391/391 [00:41<00:00,  9.46it/s]


Epoch 29/30 - Loss: 0.8907 - Val Acc: 57.63%


DKD Epoch 30/30: 100%|██████████| 391/391 [00:41<00:00,  9.43it/s]


Epoch 30/30 - Loss: 0.8871 - Val Acc: 57.77%
DKD training finished. Best Val Acc: 57.88%
Final student accuracy (DKD): 57.77
Saved best student to student_vgg11_DKD.pth


In [ ]:
# import shutil
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Local model file (after training)
# local_path = "student_vgg11_DKD.pth"

# # Destination in Google Drive
# drive_path = "/content/drive/MyDrive/student_vgg11_DKD.pth"

# # Copy file
# shutil.copy(local_path, drive_path)
# print(f"✅ Model saved to Google Drive at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_DKD.pth


##Hint Distillation

In [ ]:
# hint_distill.py
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import numpy as np
import random

# ============================================================
# 1. Utility functions
# ============================================================

def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def accuracy(output, target):
    _, preds = torch.max(output, 1)
    return torch.sum(preds == target).item() / target.size(0)


# ============================================================
# 2. Dataset and Dataloader
# ============================================================

def get_dataloaders(batch_size=128):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761))
    ])

    trainset = datasets.CIFAR100(root="./data", train=True,
                                 download=True, transform=transform_train)
    testset = datasets.CIFAR100(root="./data", train=False,
                                download=True, transform=transform_test)

    trainloader = DataLoader(trainset, batch_size=batch_size,
                             shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=batch_size,
                            shuffle=False, num_workers=2)
    return trainloader, testloader


# ============================================================
# 3. Model setup (Teacher, Student, Adapter)
# ============================================================

class HintAdapter(nn.Module):
    """1x1 conv to align student feature channels to teacher."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        return self.conv(x)


def get_teacher_student(device):
    # Teacher: pretrained VGG16
    teacher = models.vgg16(weights='IMAGENET1K_V1')
    teacher.classifier[6] = nn.Linear(4096, 100)
    teacher.load_state_dict(torch.load('teacher_vgg16.pth'))
    teacher = teacher.to(device)
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    # Student: VGG11
    student = models.vgg11()
    student.classifier[6] = nn.Linear(4096, 100)
    student = student.to(device)

    # ---- Find convolution layers dynamically ----
    teacher_conv_layers = [m for m in teacher.features if isinstance(m, nn.Conv2d)]
    student_conv_layers = [m for m in student.features if isinstance(m, nn.Conv2d)]

    # Choose middle convolution layer pair (e.g., 4th conv)
    t_layer = teacher_conv_layers[4]
    s_layer = student_conv_layers[3]

    # We'll need to hook these, so we must find their indices in .features
    t_index = list(teacher.features).index(t_layer)
    s_index = list(student.features).index(s_layer)

    # Adapter to match feature channel dims
    t_channels = t_layer.out_channels
    s_channels = s_layer.out_channels
    adapter = HintAdapter(s_channels, t_channels).to(device)

    return teacher, student, adapter, t_index, s_index


# ============================================================
# 4. Hook functions to extract feature maps
# ============================================================

def get_feature_hook(storage_dict, name):
    def hook(module, input, output):
        storage_dict[name] = output
    return hook


# ============================================================
# 5. Training loop
# ============================================================

def train_hint_kd(epochs=30, batch_size=128, lr=0.01, lambda_hint=0.5, device='cuda'):
    set_seed(42)
    trainloader, testloader = get_dataloaders(batch_size)
    teacher, student, adapter, t_layer, s_layer = get_teacher_student(device)

    # Feature storage
    feat_t, feat_s = {}, {}
    teacher.features[t_layer].register_forward_hook(get_feature_hook(feat_t, "t"))
    student.features[s_layer].register_forward_hook(get_feature_hook(feat_s, "s"))

    # Optimizer
    optimizer = optim.SGD(
        list(student.parameters()) + list(adapter.parameters()),
        lr=lr, momentum=0.9, weight_decay=5e-4
    )
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    ce_loss = nn.CrossEntropyLoss()
    mse_loss = nn.MSELoss()

    best_acc = 0
    os.makedirs("checkpoints", exist_ok=True)

    for epoch in range(epochs):
        student.train()
        running_loss = 0
        running_acc = 0

        for imgs, labels in tqdm(trainloader, desc=f"Epoch [{epoch+1}/{epochs}]"):
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.no_grad():
                _ = teacher(imgs)
            _ = student(imgs)

            # Feature distillation loss
            loss_hint = mse_loss(adapter(feat_s["s"]), feat_t["t"].detach())

            # Classification loss
            outputs = student(imgs)
            loss_ce = ce_loss(outputs, labels)

            # Combine losses
            loss = (1 - lambda_hint) * loss_ce + lambda_hint * loss_hint
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_acc += accuracy(outputs, labels)

        scheduler.step()
        avg_train_loss = running_loss / len(trainloader)
        avg_train_acc = running_acc / len(trainloader)

        # Validation
        student.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in testloader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = student(imgs)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total * 100

        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_train_loss:.4f} | "
              f"Train Acc: {avg_train_acc*100:.2f}% | Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(student.state_dict(), "student_vgg11_hints.pth")

    print(f"✅ Hint KD complete. Best Acc: {best_acc:.2f}% | "
          f"Saved at student_vgg11_hints.pth")


# ============================================================
# 6. Run directly
# ============================================================

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_hint_kd(epochs=30, batch_size=128, lr=0.01, lambda_hint=0.5, device=device)


Epoch [1/30]: 100%|██████████| 391/391 [00:49<00:00,  7.87it/s]


Epoch [1/30] | Loss: 5.2150 | Train Acc: 1.66% | Val Acc: 4.30%


Epoch [2/30]: 100%|██████████| 391/391 [00:49<00:00,  7.88it/s]


Epoch [2/30] | Loss: 4.4446 | Train Acc: 6.79% | Val Acc: 10.88%


Epoch [3/30]: 100%|██████████| 391/391 [00:49<00:00,  7.84it/s]


...

Epoch [28/30] | Loss: 1.6610 | Train Acc: 66.43% | Val Acc: 56.58%


Epoch [29/30]: 100%|██████████| 391/391 [00:49<00:00,  7.83it/s]


Epoch [29/30] | Loss: 1.6505 | Train Acc: 67.03% | Val Acc: 56.83%


Epoch [30/30]: 100%|██████████| 391/391 [00:49<00:00,  7.85it/s]


Epoch [30/30] | Loss: 1.6468 | Train Acc: 67.18% | Val Acc: 56.70%
✅ Hint KD complete. Best Acc: 56.83% | Saved at student_vgg11_hints.pth


In [ ]:
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Local model file (after training)
local_path = "student_vgg11_hints.pth"

# Destination in Google Drive
drive_path = "/content/drive/MyDrive/student_vgg11_hints.pth"

# Copy file
shutil.copy(local_path, drive_path)
print(f"✅ Model saved to Google Drive at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_hints.pth


##Contrastive Representation Distillation (CRD)

In [ ]:
# crd_distill_run.py
# Contrastive Representation Distillation (in-batch InfoNCE) for VGG16 -> VGG11 on CIFAR-100
# Saves best student as: student_vgg11_CRD.pth
# Logs to: crd_training_log.csv

import os
import random
import csv
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg11, vgg16

# -----------------------
# Config / Hyperparams
# -----------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_WORKERS = 4
NUM_EPOCHS = 30
NUM_CLASSES = 100
LR = 0.01
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
CLIP_GRAD = 5.0
SEED = 42
PRINT_EVERY = 1

# CRD-specific
PROJ_DIM = 256        # projection embedding dimension
TAU = 0.07            # contrastive temperature
LAMBDA_CRD = 1.0      # weight for contrastive loss vs CE

TEACHER_CKPT = "/content/drive/MyDrive/teacher_vgg16.pth"
STUDENT_OUT = "student_vgg11_CRD.pth"
LOG_CSV = "crd_training_log.csv"

# -----------------------
# Utilities
# -----------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    # deterministic behavior (may slow training)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# -----------------------
# Data
# -----------------------
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),(0.2675, 0.2565, 0.2761))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),(0.2675, 0.2565, 0.2761))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# -----------------------
# Model builders
# -----------------------
def build_teacher(num_classes=NUM_CLASSES):
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def build_student(num_classes=NUM_CLASSES):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

# -----------------------
# Feature hook helper
# -----------------------
def register_last_feature_hook(model, storage_dict, key="feat"):
    """
    Registers a forward hook on the last conv layer in model.features to capture the conv feature map.
    """
    last_conv = None
    for m in model.features:
        if isinstance(m, nn.Conv2d):
            last_conv = m
    if last_conv is None:
        raise RuntimeError("No Conv2d found in model.features")
    handle = last_conv.register_forward_hook(lambda module, inp, out: storage_dict.update({key: out}))
    return handle

# -----------------------
# Projection head
# -----------------------
class ProjectionHead(nn.Module):
    def __init__(self, in_dim, proj_dim=PROJ_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.ReLU(inplace=True),
            nn.Linear(in_dim, proj_dim)
        )
    def forward(self, x):
        return self.net(x)

# -----------------------
# In-batch InfoNCE loss
# -----------------------
def info_nce_loss(student_proj, teacher_proj, tau=TAU):
    """
    student_proj: (B, D)
    teacher_proj: (B, D)
    returns scalar loss (average over batch)
    """
    student_norm = F.normalize(student_proj, dim=1, p=2)
    teacher_norm = F.normalize(teacher_proj, dim=1, p=2)
    logits = torch.matmul(student_norm, teacher_norm.t())  # (B, B)
    logits = logits / tau
    targets = torch.arange(logits.size(0), device=logits.device)
    loss = F.cross_entropy(logits, targets)
    return loss

# -----------------------
# Evaluation
# -----------------------
def evaluate(model, dataloader, device=DEVICE):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            out = model(imgs)
            preds = out.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

# -----------------------
# Training loop
# -----------------------
def train_crd(teacher, student, trainloader, testloader, num_epochs=NUM_EPOCHS, device=DEVICE):
    teacher.eval()
    # feature storage dicts
    feat_t = {}
    feat_s = {}

    # register hooks
    h_t = register_last_feature_hook(teacher, feat_t, key="feat")
    h_s = register_last_feature_hook(student, feat_s, key="feat")

    # avgpool for conv maps -> vector
    avgpool = nn.AdaptiveAvgPool2d((1,1)).to(device)

    # determine dims via single forward
    with torch.no_grad():
        dummy = torch.randn(1,3,32,32).to(device)
        teacher(dummy)
        t_map = feat_t["feat"]  # (1, C_t, H, W)
        t_dim = t_map.shape[1]
        student(dummy)
        s_map = feat_s["feat"]
        s_dim = s_map.shape[1]

    # projectors
    proj_t = ProjectionHead(t_dim, proj_dim=PROJ_DIM).to(device)
    proj_s = ProjectionHead(s_dim, proj_dim=PROJ_DIM).to(device)

    # Option: initialize proj_t from identity-ish or leave random. We'll freeze proj_t.
    for p in proj_t.parameters():
        p.requires_grad = False

    # optimizer: student params + proj_s params
    optim_params = list(student.parameters()) + list(proj_s.parameters())
    optimizer = optim.SGD(optim_params, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    ce_loss = nn.CrossEntropyLoss()

    best_acc = 0.0
    os.makedirs("checkpoints", exist_ok=True)

    # Prepare CSV log
    with open(LOG_CSV, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["epoch", "train_loss", "train_ce", "train_crd", "val_acc"])

    for epoch in range(1, num_epochs+1):
        student.train()
        proj_s.train()
        running_loss = 0.0
        running_ce = 0.0
        running_crd = 0.0
        iters = 0
        pbar = tqdm(trainloader, desc=f"CRD Epoch {epoch}/{num_epochs}")
        for imgs, labels in pbar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            # teacher forward (no grad)
            with torch.no_grad():
                _ = teacher(imgs)
                t_map = feat_t["feat"]        # (B, C_t, H, W)
                t_vec = avgpool(t_map).view(imgs.size(0), -1)  # (B, C_t)

            # student forward (grad enabled)
            _ = student(imgs)
            s_map = feat_s["feat"]           # (B, C_s, H, W)
            s_vec = avgpool(s_map).view(imgs.size(0), -1)  # (B, C_s)

            # project
            with torch.no_grad():
                t_proj = proj_t(t_vec)  # frozen teacher proj (B, PROJ_DIM)
            s_proj = proj_s(s_vec)      # (B, PROJ_DIM)

            # compute losses
            out_logits = student(imgs)   # forward again for classifier outputs (could reuse earlier but safe)
            loss_ce = ce_loss(out_logits, labels)
            loss_crd = info_nce_loss(s_proj, t_proj.detach(), tau=TAU)
            loss = loss_ce + LAMBDA_CRD * loss_crd

            optimizer.zero_grad()
            loss.backward()
            # clip grads
            torch.nn.utils.clip_grad_norm_(optim_params, CLIP_GRAD)
            optimizer.step()

            running_loss += float(loss.item())
            running_ce += float(loss_ce.item())
            running_crd += float(loss_crd.item())
            iters += 1
            pbar.set_postfix(train_loss=running_loss/iters, ce=running_ce/iters, crd=running_crd/iters)

        scheduler.step()

        # evaluate
        val_acc = evaluate(student, testloader, device=device)
        avg_loss = running_loss / max(1, iters)
        avg_ce = running_ce / max(1, iters)
        avg_crd = running_crd / max(1, iters)

        # log to csv
        with open(LOG_CSV, "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([epoch, avg_loss, avg_ce, avg_crd, val_acc])

        if epoch % PRINT_EVERY == 0:
            print(f"Epoch {epoch}/{num_epochs} - loss: {avg_loss:.4f} | CE: {avg_ce:.4f} | CRD: {avg_crd:.4f} | val_acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(student.state_dict(), STUDENT_OUT)
            print(f"  -> New best saved: {STUDENT_OUT} (acc {best_acc:.2f}%)")

    # remove hooks: not strictly necessary in script exit
    try:
        h_t.remove()
        h_s.remove()
    except:
        pass

    print("CRD training finished. Best Val Acc: {:.2f}%".format(best_acc))
    return best_acc

# -----------------------
# Main
# -----------------------
def main():
    if not os.path.exists(TEACHER_CKPT):
        raise FileNotFoundError(f"Teacher checkpoint not found: {TEACHER_CKPT}. Train teacher first or update TEACHER_CKPT path.")

    # build and load teacher
    teacher = build_teacher(NUM_CLASSES).to(DEVICE)
    teacher.load_state_dict(torch.load(TEACHER_CKPT, map_location=DEVICE))
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    # build student
    student = build_student(NUM_CLASSES).to(DEVICE)

    print("Starting CRD training: teacher loaded, student initialized.")
    start = time.time()
    best_acc = train_crd(teacher, student, trainloader, testloader, num_epochs=NUM_EPOCHS, device=DEVICE)
    elapsed = time.time() - start
    print(f"Total time: {elapsed/60.0:.2f} minutes. Best Val Acc: {best_acc:.2f}")

    final_acc = evaluate(student, testloader, device=DEVICE)
    print("Final student accuracy (CRD): {:.2f}".format(final_acc))
    print("Best student saved to:", STUDENT_OUT)

if __name__ == "__main__":
    main()


Starting CRD training: teacher loaded, student initialized.


CRD Epoch 1/30: 100%|██████████| 391/391 [00:53<00:00,  7.28it/s, ce=4.11, crd=3.5, train_loss=7.61]


Epoch 1/30 - loss: 7.6112 | CE: 4.1086 | CRD: 3.5025 | val_acc: 14.85%
  -> New best saved: student_vgg11_CRD.pth (acc 14.85%)


CRD Epoch 2/30: 100%|██████████| 391/391 [00:54<00:00,  7.12it/s, ce=3.45, crd=2.58, train_loss=6.02]


Epoch 2/30 - loss: 6.0226 | CE: 3.4470 | CRD: 2.5756 | val_acc: 22.70%
  -> New best saved: student_vgg11_CRD.pth (acc 22.70%)


...

Epoch 28/30 - loss: 1.3602 | CE: 0.8883 | CRD: 0.4718 | val_acc: 61.26%
  -> New best saved: student_vgg11_CRD.pth (acc 61.26%)


CRD Epoch 29/30: 100%|██████████| 391/391 [00:56<00:00,  6.89it/s, ce=0.88, crd=0.47, train_loss=1.35]


Epoch 29/30 - loss: 1.3502 | CE: 0.8800 | CRD: 0.4702 | val_acc: 61.14%


CRD Epoch 30/30: 100%|██████████| 391/391 [00:56<00:00,  6.93it/s, ce=0.873, crd=0.47, train_loss=1.34]


Epoch 30/30 - loss: 1.3423 | CE: 0.8728 | CRD: 0.4695 | val_acc: 61.16%
CRD training finished. Best Val Acc: 61.26%
Total time: 31.53 minutes. Best Val Acc: 61.26
Final student accuracy (CRD): 61.16
Best student saved to: student_vgg11_CRD.pth


In [ ]:
# import shutil
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# # Local model file (after training)
# local_path = "student_vgg11_CRD.pth"

# # Destination in Google Drive
# drive_path = "/content/drive/MyDrive/student_vgg11_CRD.pth"

# # Copy file
# shutil.copy(local_path, drive_path)
# print(f"✅ Model saved to Google Drive at: {drive_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive at: /content/drive/MyDrive/student_vgg11_CRD.pth


##Comparing Probability Distributions

In [ ]:
# ==============================================================
# Part 3: Compare Teacher–Student Output Distributions (KL Divergence)
# ==============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg11_bn, vgg11, vgg16
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

# --------------------------------------------------------------
# Model definitions
# --------------------------------------------------------------
def get_vgg16_teacher(num_classes=100):
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def get_vgg11_student(num_classes=100):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def get_vgg11bn_student(num_classes=100):
    model = vgg11_bn(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

# --------------------------------------------------------------
# CIFAR-100 dataloaders
# --------------------------------------------------------------
def get_dataloaders(batch_size=64):
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761))
    ])
    testset = torchvision.datasets.CIFAR100(root="./data", train=False,
                                            download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return testloader

# --------------------------------------------------------------
# Load teacher and student models
# --------------------------------------------------------------
def load_models(device):
    base_path = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content"

    teacher = get_vgg16_teacher().to(device)
    teacher.load_state_dict(torch.load(os.path.join(base_path, "teacher_vgg16.pth"), map_location=device))
    teacher.eval()

    # Define students
    students = {
        "Independent": get_vgg11_student().to(device),
        "LogitMatching": get_vgg11_student().to(device),
        "Hint": get_vgg11_student().to(device),
        "DKD": get_vgg11_student().to(device),
        "LabelSmoothing": get_vgg11bn_student().to(device)
    }

    ckpts = {
        "Independent": os.path.join(base_path, "student_vgg11_independent.pth"),
        "LogitMatching": os.path.join(base_path, "student_vgg11_LM_tuned.pth"),
        "Hint": os.path.join(base_path, "student_vgg11_hints.pth"),
        "DKD": os.path.join(base_path, "student_vgg11_DKD.pth"),
        "LabelSmoothing": os.path.join(base_path, "student_vgg11_LS.pth"),
    }

    for name, model in students.items():
        ckpt_path = ckpts[name]
        if os.path.exists(ckpt_path):
            model.load_state_dict(torch.load(ckpt_path, map_location=device))
            model.eval()
            print(f"✅ Loaded {name} model from {ckpt_path}")
        else:
            print(f"⚠️ Warning: checkpoint not found for {name}: {ckpt_path}")

    print("✅ All available models loaded.")
    return teacher, students

# --------------------------------------------------------------
# KL divergence (numerically stable)
# --------------------------------------------------------------
def compute_kl_divergence(p_teacher, p_student):
    kl = F.kl_div(p_student.log(), p_teacher, reduction="batchmean")
    return kl.item()

# --------------------------------------------------------------
# Compare teacher and students
# --------------------------------------------------------------
def compare_distributions(device, batch_size=64, temperature=4.0, num_batches=20):
    testloader = get_dataloaders(batch_size)
    teacher, students = load_models(device)

    results = {k: [] for k in students.keys()}

    with torch.no_grad():
        for i, (images, _) in enumerate(tqdm(testloader, total=num_batches, desc="Evaluating KL Divergence")):
            if i >= num_batches:
                break

            images = images.to(device)
            # Teacher probs (softened)
            t_logits = teacher(images) / temperature
            t_probs = F.softmax(t_logits, dim=1)

            for name, student in students.items():
                s_logits = student(images) / temperature
                s_probs = F.softmax(s_logits, dim=1)

                # Compute divergence between teacher & student
                kl = compute_kl_divergence(t_probs, s_probs)
                results[name].append(kl)

    avg_kl = {name: np.mean(vals) for name, vals in results.items() if len(vals) > 0}

    print("\n=== 📊 Average KL Divergence: Teacher vs Students ===")
    for name, val in avg_kl.items():
        print(f"{name:15s}: {val:.6f}")

    # Save results
    df = pd.DataFrame(list(avg_kl.items()), columns=["Student Model", "KL Divergence"])
    df.sort_values("KL Divergence", inplace=True)
    out_path = "KL_results.csv"
    df.to_csv(out_path, index=False)
    print(f"\n📁 Results saved to: {out_path}")

    return df

# --------------------------------------------------------------
# Run
# --------------------------------------------------------------
if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    results_df = compare_distributions(device, batch_size=64, temperature=4.0, num_batches=20)
    print("\n✅ KL Divergence Comparison Complete.")
    display(results_df)

⚠️ Warning: checkpoint not found for Independent: /content/drive/MyDrive/student_vgg11_independent.pth
✅ Loaded LogitMatching model from /content/drive/MyDrive/student_vgg11_LM_tuned.pth
⚠️ Warning: checkpoint not found for Hint: /content/drive/MyDrive/student_vgg11_hints.pth
✅ Loaded DKD model from /content/drive/MyDrive/student_vgg11_DKD.pth
⚠️ Warning: checkpoint not found for LabelSmoothing: /content/drive/MyDrive/student_vgg11_LS.pth
✅ All available models loaded.


Evaluating KL Divergence: 100%|██████████| 20/20 [00:01<00:00, 10.71it/s]


=== 📊 Average KL Divergence: Teacher vs Students ===
Independent    : 0.469650
LogitMatching  : 0.061759
Hint           : 0.469483
DKD            : 0.080693
LabelSmoothing : 0.486313

📁 Results saved to: KL_results.csv

✅ KL Divergence Comparison Complete.


,Student Model,KL Divergence
1,LogitMatching,0.061759
3,DKD,0.080693
2,Hint,0.469483
0,Independent,0.469650
4,LabelSmoothing,0.486313


##Examining Localization Knowledge Transfer

In [ ]:
# !pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=80d311dd2a778b103dc00cdf2cd771ba362b3a4f303fcf777b21125ddac29a52
  Stored in directory: /root/.cache/pip/wheels/fb/3b/09/2afc520f3d69bc26ae6bd87416759c820a3f7d05c1a077bbf6
Successfully built grad-cam


In [ ]:
# gradcam_kd_compare.py
# Run in Colab or locally. Requires: torch, torchvision, pytorch-grad-cam, matplotlib, pandas, tqdm
# Installs: pip install grad-cam

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.models import vgg11, vgg11_bn, vgg16
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import pandas as pd
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# ---------------- Config ----------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
N_IMAGES = 20               # number of query images to analyze
OUTPUT_DIR = "gradcam_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Candidate checkpoint filenames (will search in drive then cwd)
CKPT_NAMES = {
    "Teacher": "teacher_vgg16.pth",
    "SI": "student_vgg11_SI.pth",
    "LM": "student_vgg11_LM_tuned.pth",
    "Hints": "student_vgg11_hints.pth",
    "DKD": "student_vgg11_DKD.pth",
    "CRD": "student_vgg11_CRD.pth",   # optional
    "LS": "student_vgg11_LS.pth"      # optional
}

# CIFAR-100 normalization used during training
MEAN = (0.5071, 0.4867, 0.4408)
STD  = (0.2675, 0.2565, 0.2761)

# IoU top fraction
TOPK_FRAC = 0.20

# Input size for Grad-CAM overlays (we use 224 to work with VGG models)
INP_SIZE = 224

# ---------------- Helpers ----------------
def find_checkpoint(filename):
    # prefer Google Drive path if mounted, else current dir
    drive_path = "/content/drive/MyDrive"
    candidates = []
    if os.path.exists(drive_path):
        candidates.append(os.path.join(drive_path, filename))
    candidates.append(filename)
    for p in candidates:
        if os.path.exists(p):
            return p
    return None

def build_teacher(num_classes=100):
    m = vgg16(weights=None)
    m.classifier[6] = nn.Linear(4096, num_classes)
    return m

def build_student(num_classes=100, bn=False):
    if bn:
        m = vgg11_bn(weights=None)
    else:
        m = vgg11(weights=None)
    m.classifier[6] = nn.Linear(4096, num_classes)
    return m

def find_last_conv_layer(model):
    last_conv = None
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            last_conv = m
    if last_conv is None:
        raise ValueError("No Conv2d found in model")
    return last_conv

def normalize_map(cam):
    cam = np.array(cam, dtype=np.float32)
    cam = cam - cam.min()
    if cam.max() > 0:
        cam = cam / (cam.max() + 1e-12)
    return cam

def pearson_corr(a, b):
    a = a.flatten(); b = b.flatten()
    a = (a - a.mean()) / (a.std() + 1e-12)
    b = (b - b.mean()) / (b.std() + 1e-12)
    return float(np.mean(a * b))

def cosine_sim(a, b):
    a = a.flatten(); b = b.flatten()
    an = np.linalg.norm(a); bn = np.linalg.norm(b)
    if an < 1e-12 or bn < 1e-12:
        return 0.0
    return float(np.dot(a, b) / (an * bn))

def iou_topk(a, b, frac=TOPK_FRAC):
    flat_a = a.flatten(); flat_b = b.flatten()
    k = max(1, int(len(flat_a) * frac))
    top_a = np.argpartition(-flat_a, k-1)[:k]
    top_b = np.argpartition(-flat_b, k-1)[:k]
    set_a = set(top_a.tolist()); set_b = set(top_b.tolist())
    inter = len(set_a & set_b); union = len(set_a | set_b)
    if union == 0:
        return 0.0
    return inter / union

# ---------------- Prepare dataset ----------------
transform_vis = transforms.Compose([
    transforms.Resize(INP_SIZE),
    transforms.ToTensor()
])
transform_input = transforms.Compose([
    transforms.Resize(INP_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

test_vis = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_vis)
test_input = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_input)

# select indices: first N_IMAGES by default (you can change)
selected_indices = list(range(N_IMAGES))
selected_vis = [test_vis[i][0] for i in selected_indices]
selected_labels = [test_input[i][1] for i in selected_indices]  # ground truth if needed

# ---------------- Load models ----------------
print("Loading models (searching in drive then cwd)...")
models = {}
for role, fname in CKPT_NAMES.items():
    path = find_checkpoint(fname)
    if path is None:
        print(f"  - {role}: checkpoint not found ({fname}) — skipping")
        continue
    if role == "Teacher":
        model = build_teacher().to(DEVICE)
    else:
        # label smoothing model used vgg11_bn in some runs
        if role == "LS":
            model = build_student(bn=True).to(DEVICE)
        else:
            model = build_student().to(DEVICE)
    try:
        state = torch.load(path, map_location=DEVICE)
        model.load_state_dict(state)
        model.eval()
        models[role] = model
        print(f"  - {role}: loaded from {path}")
    except Exception as e:
        print(f"  - {role}: failed to load {path}: {e}")

if "Teacher" not in models:
    raise RuntimeError("Teacher checkpoint is required. Place teacher_vgg16.pth in drive or cwd.")

print(f"Models available for Grad-CAM: {list(models.keys())}")

# ---------------- Prepare CAM objects ----------------
cams = {}
for name, m in models.items():
    layer = find_last_conv_layer(m)
    # Updated: Use the device directly instead of the use_cuda argument
    cam_obj = GradCAM(model=m, target_layers=[layer])
    cams[name] = cam_obj

# ---------------- Compute GradCAMs & metrics ----------------
rows = []
for idx in range(len(selected_vis)):
    vis_img = selected_vis[idx]  # tensor (3,H,W) in [0,1]
    np_img = np.transpose(vis_img.numpy(), (1,2,0))
    np_img = np.clip(np_img, 0, 1)

    input_tensor = preprocess_image(np_img, mean=MEAN, std=STD).to(DEVICE)

    # Teacher prediction / target class
    with torch.no_grad():
        t_logits = models["Teacher"](input_tensor)
        t_pred = int(torch.argmax(t_logits, dim=1).item())

    target = [ClassifierOutputTarget(t_pred)]
    teacher_cam = cams["Teacher"](input_tensor=input_tensor, targets=target, eigen_smooth=False, aug_smooth=False)
    teacher_map = normalize_map(teacher_cam[0])

    # make output dir for image
    out_dir = os.path.join(OUTPUT_DIR, f"img_{idx:03d}")
    os.makedirs(out_dir, exist_ok=True)

    # Save teacher overlay
    teacher_overlay = show_cam_on_image(np_img, teacher_map, use_rgb=True)
    Image.fromarray(teacher_overlay).save(os.path.join(out_dir, "teacher_overlay.png"))
    np.save(os.path.join(out_dir, "teacher_map.npy"), teacher_map)

    row = {"image_idx": idx, "teacher_pred": t_pred}

    # For each student model compute cam and metrics vs teacher_map
    for name, cam_obj in cams.items():
        if name == "Teacher":
            continue
        try:
            student_cam = cam_obj(input_tensor=input_tensor, targets=target, eigen_smooth=False, aug_smooth=False)
        except Exception as e:
            print(f"Warning: CAM failed for {name} on img {idx}: {e}")
            student_map = np.zeros_like(teacher_map)
        else:
            student_map = normalize_map(student_cam[0])

        # metrics
        p_corr = pearson_corr(teacher_map, student_map)
        c_sim = cosine_sim(teacher_map, student_map)
        iou = iou_topk(teacher_map, student_map, frac=TOPK_FRAC)

        row[f"{name}_pearson"] = p_corr
        row[f"{name}_cosine"]  = c_sim
        row[f"{name}_iou"]     = iou

        # save overlay and map
        overlay = show_cam_on_image(np_img, student_map, use_rgb=True)
        Image.fromarray(overlay).save(os.path.join(out_dir, f"{name}_overlay.png"))
        np.save(os.path.join(out_dir, f"{name}_map.npy"), student_map)

    rows.append(row)
    print(f"Processed image {idx+1}/{len(selected_vis)}")

# ---------------- Save CSV of metrics ----------------
df = pd.DataFrame(rows)
csv_path = os.path.join(OUTPUT_DIR, "gradcam_similarity_metrics.csv")
df.to_csv(csv_path, index=False)
print("Saved metrics to:", csv_path)

# ---------------- Aggregate & plot ----------------
# compute mean metrics per model
student_names = [n for n in cams.keys() if n != "Teacher"]
summary = {}
for name in student_names:
    pear = df[f"{name}_pearson"].mean()
    cos  = df[f"{name}_cosine"].mean()
    iou  = df[f"{name}_iou"].mean()
    summary[name] = {"pearson": pear, "cosine": cos, "iou": iou}
summary_df = pd.DataFrame.from_dict(summary, orient="index")
summary_df = summary_df[["pearson", "cosine", "iou"]]

print("\nMean Grad-CAM similarity metrics (Teacher vs Student):\n", summary_df)

# save summary csv
summary_csv = os.path.join(OUTPUT_DIR, "gradcam_similarity_summary.csv")
summary_df.to_csv(summary_csv)
print("Saved summary CSV to:", summary_csv)

# plot bar charts for each metric
plt.figure(figsize=(10,5))
summary_df["pearson"].plot(kind="bar")
plt.title("Mean Pearson Correlation (Teacher vs Student CAM)")
plt.ylabel("Pearson")
plt.ylim(-1,1)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "pearson_summary.png"))
plt.close()

plt.figure(figsize=(10,5))
summary_df["cosine"].plot(kind="bar")
plt.title("Mean Cosine Similarity (Teacher vs Student CAM)")
plt.ylabel("Cosine")
plt.ylim(0,1)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "cosine_summary.png"))
plt.close()

plt.figure(figsize=(10,5))
summary_df["iou"].plot(kind="bar")
plt.title(f"Mean IoU (top {int(TOPK_FRAC*100)}%) (Teacher vs Student CAM)")
plt.ylabel("IoU")
plt.ylim(0,1)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "iou_summary.png"))
plt.close()

print("Saved summary plots in", OUTPUT_DIR)

# ---------------- Optionally create montage per image ----------------
# saves a side-by-side composite for quick inspection
for idx in range(len(selected_vis)):
    out_dir = os.path.join(OUTPUT_DIR, f"img_{idx:03d}")
    imgs = []
    # teacher first
    teacher_path = os.path.join(out_dir, "teacher_overlay.png")
    if os.path.exists(teacher_path):
        imgs.append(Image.open(teacher_path))
    student_names = [n for n in CKPT_NAMES.keys() if n != "Teacher" and n in models.keys()] # Use available models
    for name in student_names:
        p = os.path.join(out_dir, f"{name}_overlay.png")
        if os.path.exists(p):
            imgs.append(Image.open(p))
    # create horizontal montage
    if imgs: # check if there are any images to montage
        widths, heights = zip(*(i.size for i in imgs))
        total_w = sum(widths)
        max_h = max(heights)
        montage = Image.new('RGB', (total_w, max_h))
        x = 0
        for im in imgs:
            montage.paste(im, (x, 0))
            x += im.size[0]
        montage.save(os.path.join(out_dir, "montage.png"))
    else:
        print(f"No images found to create montage for image index {idx}")

print("Montages saved. All done.")

Loading models (searching in drive then cwd)...
  - Teacher: loaded from /content/drive/MyDrive/teacher_vgg16.pth
  - SI: loaded from /content/drive/MyDrive/student_vgg11_SI.pth
  - LM: loaded from /content/drive/MyDrive/student_vgg11_LM_tuned.pth
  - Hints: checkpoint not found (student_vgg11_hints.pth) — skipping
  - DKD: loaded from /content/drive/MyDrive/student_vgg11_DKD.pth
  - CRD: loaded from /content/drive/MyDrive/student_vgg11_CRD.pth
  - LS: checkpoint not found (student_vgg11_LS.pth) — skipping
Models available for Grad-CAM: ['Teacher', 'SI', 'LM', 'DKD', 'CRD']
Processed image 1/20
Processed image 2/20
Processed image 3/20
Processed image 4/20
Processed image 5/20
Processed image 6/20
Processed image 7/20
Processed image 8/20
Processed image 9/20
Processed image 10/20
Processed image 11/20
Processed image 12/20
Processed image 13/20
Processed image 14/20
Processed image 15/20
Processed image 16/20
Processed image 17/20
Processed image 18/20
Processed image 19/20
Processed 

##Checking for Color Invariance with CRD

In [ ]:
# crd_color_invariance.py
# Part 5: Check Color Invariance with CRD
# 1) Finetune teacher with color jitter augmentations
# 2) Use that teacher to train a CRD student (student uses regular augmentations)
# 3) Evaluate teacher and student on original and color-jittered test sets
#
# Requirements: torch, torchvision, tqdm, numpy, pandas
# Place teacher_vgg16.pth in working dir or /content/drive/MyDrive/

import os
import time
import random
import csv
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16, vgg11
from torch.utils.data import DataLoader

# ---------------------------
# Config / Hyperparams
# ---------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BASE_PATH = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "."

# Files (existing and outputs)
TEACHER_CKPT = os.path.join(BASE_PATH, "teacher_vgg16.pth")            # required (existing trained teacher)
TEACHER_COLOR_CKPT = os.path.join(BASE_PATH, "teacher_vgg16_colorfinetune.pth")
STUDENT_CRD_OUT = os.path.join(BASE_PATH, "student_vgg11_CRD_colorteacher.pth")
LOG_CSV = os.path.join(BASE_PATH, "crd_color_invariance_log.csv")

# Training params
FINETUNE_TEACHER = True              # whether to finetune teacher with color augmentations
FINETUNE_EPOCHS = 8                  # short finetune (adjust if you want)
FINETUNE_LR = 0.01

DO_TRAIN_CRD = True                  # whether to train student via CRD using the color teacher
CRD_EPOCHS = 30
BATCH_SIZE = 128
NUM_WORKERS = 2
LR_CRD = 0.01
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9
CLIP_GRAD = 5.0

# CRD hyperparams
PROJ_DIM = 256
TAU = 0.07
LAMBDA_CRD = 1.0

NUM_CLASSES = 100
SEED = 42

# ---------------------------
# Reproducibility
# ---------------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# ---------------------------
# Model builders
# ---------------------------
def build_teacher(num_classes=NUM_CLASSES):
    model = vgg16(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

def build_student(num_classes=NUM_CLASSES):
    model = vgg11(weights=None)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model

# ---------------------------
# Datasets & transforms
# ---------------------------
# Base normalization (same as your previous experiments)
MEAN = (0.5071, 0.4867, 0.4408)
STD  = (0.2675, 0.2565, 0.2761)

# Teacher finetune augmentation: strong color jitter + random grayscale
teacher_train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.1),
    transforms.RandomApply([transforms.RandomGrayscale(p=0.2)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

# Student training uses regular augmentations (we do NOT want to explicitly teach student color invariance)
student_train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

# Test transforms: original (no jitter) and color-jittered test
test_transform_orig = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

# Color-jittered test uses same color augment + to-tensor + normalize (deterministic variation per sample)
test_transform_color = transforms.Compose([
    transforms.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.1),
    transforms.RandomApply([transforms.RandomGrayscale(p=0.2)], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

# Data loaders
def get_dataloaders(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, for_teacher=False):
    if for_teacher:
        train_transform = teacher_train_transform
    else:
        train_transform = student_train_transform

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

    testset_orig = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform_orig)
    testloader_orig = DataLoader(testset_orig, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    testset_color = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform_color)
    testloader_color = DataLoader(testset_color, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return trainloader, testloader_orig, testloader_color

# ---------------------------
# Utilities: evaluate
# ---------------------------
def evaluate_model(model, dataloader, device=DEVICE):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            out = model(imgs)
            preds = out.argmax(dim=1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return 100.0 * correct / total

# ---------------------------
# Finetune teacher with color augmentation
# ---------------------------
def finetune_teacher_color(teacher_ckpt, out_ckpt, epochs=FINETUNE_EPOCHS, lr=FINETUNE_LR):
    if not os.path.exists(teacher_ckpt):
        raise FileNotFoundError(f"Teacher checkpoint not found at {teacher_ckpt}. Provide pretrained teacher first.")

    # load teacher
    teacher = build_teacher().to(DEVICE)
    teacher.load_state_dict(torch.load(teacher_ckpt, map_location=DEVICE))
    print("Loaded teacher from", teacher_ckpt)

    # freeze most layers? here we finetune entire model (small lr)
    for p in teacher.parameters():
        p.requires_grad = True

    trainloader, test_orig, test_color = get_dataloaders(for_teacher=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(teacher.parameters(), lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    best_acc_color = 0.0
    for epoch in range(1, epochs+1):
        teacher.train()
        running_loss = 0.0
        pbar = tqdm(trainloader, desc=f"Finetune Teacher (color) Epoch {epoch}/{epochs}")
        for imgs, labels in pbar:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            out = teacher(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            running_loss += float(loss.item())
            pbar.set_postfix(avg_loss=running_loss/(len(pbar)+1))
        scheduler.step()

        # Evaluate on color test set to measure color invariance of teacher
        teacher.eval()
        acc_color = evaluate_model(teacher, test_color)
        acc_orig = evaluate_model(teacher, test_orig)
        print(f"Epoch {epoch}: Teacher val orig: {acc_orig:.2f}% | val color: {acc_color:.2f}%")
        if acc_color > best_acc_color:
            best_acc_color = acc_color
            torch.save(teacher.state_dict(), out_ckpt)
            print(f"  -> Saved color-finetuned teacher to {out_ckpt} (val_color {best_acc_color:.2f}%)")

    print("Teacher finetune complete. Best color test acc:", best_acc_color)
    return out_ckpt

# ---------------------------
# CRD training (student uses regular augmentations)
# ---------------------------
# We'll reuse the CRD implementation with last conv features + projection heads
def register_last_feature_hook(model, storage_dict, key="feat"):
    last_conv = None
    for m in model.features:
        if isinstance(m, nn.Conv2d):
            last_conv = m
    if last_conv is None:
        raise RuntimeError("No Conv2d found in model.features")
    handle = last_conv.register_forward_hook(lambda module, inp, out: storage_dict.update({key: out}))
    return handle

class ProjectionHead(nn.Module):
    def __init__(self, in_dim, proj_dim=PROJ_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.ReLU(inplace=True),
            nn.Linear(in_dim, proj_dim)
        )
    def forward(self, x):
        return self.net(x)

def info_nce_loss(student_proj, teacher_proj, tau=TAU):
    student_norm = F.normalize(student_proj, dim=1)
    teacher_norm = F.normalize(teacher_proj, dim=1)
    logits = torch.matmul(student_norm, teacher_norm.t()) / tau
    targets = torch.arange(logits.size(0), device=logits.device)
    loss = F.cross_entropy(logits, targets)
    return loss

def train_crd_with_color_teacher(teacher_ckpt, out_student_ckpt, epochs=CRD_EPOCHS):
    # load teacher (color finetuned)
    if not os.path.exists(teacher_ckpt):
        raise FileNotFoundError(f"Color finetuned teacher not found at {teacher_ckpt}. Run finetune first.")
    teacher = build_teacher().to(DEVICE)
    teacher.load_state_dict(torch.load(teacher_ckpt, map_location=DEVICE))
    teacher.eval()
    for p in teacher.parameters():
        p.requires_grad = False

    student = build_student().to(DEVICE)

    feat_t = {}
    feat_s = {}
    h_t = register_last_feature_hook(teacher, feat_t, key="feat")
    h_s = register_last_feature_hook(student, feat_s, key="feat")
    avgpool = nn.AdaptiveAvgPool2d((1,1)).to(DEVICE)

    # single forward to get dims
    with torch.no_grad():
        dummy = torch.randn(1,3,32,32).to(DEVICE)
        teacher(dummy)
        t_map = feat_t["feat"]; t_dim = t_map.shape[1]
        student(dummy)
        s_map = feat_s["feat"]; s_dim = s_map.shape[1]

    proj_t = ProjectionHead(t_dim, proj_dim=PROJ_DIM).to(DEVICE)
    proj_s = ProjectionHead(s_dim, proj_dim=PROJ_DIM).to(DEVICE)
    # freeze teacher proj (treat teacher projections as anchors)
    for p in proj_t.parameters():
        p.requires_grad = False

    # optimizer for student + proj_s
    optimizer = optim.SGD(list(student.parameters()) + list(proj_s.parameters()), lr=LR_CRD, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    trainloader, test_orig, test_color = get_dataloaders()   # student train uses regular augmentations

    ce = nn.CrossEntropyLoss()
    best_val_color = 0.0
    # create log CSV header
    with open(LOG_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "train_ce", "train_crd", "val_orig_acc", "val_color_acc"])

    for epoch in range(1, epochs+1):
        student.train()
        proj_s.train()
        running_loss = 0.0
        running_ce = 0.0
        running_crd = 0.0
        iters = 0
        pbar = tqdm(trainloader, desc=f"CRD student training epoch {epoch}/{epochs}")
        for imgs, labels in pbar:
            imgs = imgs.to(DEVICE); labels = labels.to(DEVICE)

            # teacher forward (no grad)
            with torch.no_grad():
                _ = teacher(imgs)
                t_map = feat_t["feat"]
                t_vec = avgpool(t_map).view(imgs.size(0), -1)

            # student forward
            _ = student(imgs)
            s_map = feat_s["feat"]
            s_vec = avgpool(s_map).view(imgs.size(0), -1)

            # projections
            with torch.no_grad():
                t_proj = proj_t(t_vec)
            s_proj = proj_s(s_vec)

            # losses
            logits = student(imgs)
            loss_ce = ce(logits, labels)
            loss_crd = info_nce_loss(s_proj, t_proj.detach(), tau=TAU)
            loss = loss_ce + LAMBDA_CRD * loss_crd

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(list(student.parameters()) + list(proj_s.parameters()), CLIP_GRAD)
            optimizer.step()

            running_loss += float(loss.item())
            running_ce += float(loss_ce.item())
            running_crd += float(loss_crd.item())
            iters += 1
            pbar.set_postfix(avg_loss=running_loss/iters, ce=running_ce/iters, crd=running_crd/iters)
        scheduler.step()

        # evaluate on both original & color test sets
        val_orig_acc = evaluate_model(student, test_orig, device=DEVICE)
        val_color_acc = evaluate_model(student, test_color, device=DEVICE)
        avg_loss = running_loss / max(1, iters)
        avg_ce = running_ce / max(1, iters)
        avg_crd = running_crd / max(1, iters)

        # log
        with open(LOG_CSV, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([epoch, avg_loss, avg_ce, avg_crd, val_orig_acc, val_color_acc])

        print(f"Epoch {epoch}: val_orig: {val_orig_acc:.2f}% | val_color: {val_color_acc:.2f}%")

        if val_color_acc > best_val_color:
            best_val_color = val_color_acc
            torch.save(student.state_dict(), out_student_ckpt)
            print(f"  -> Saved best student to {out_student_ckpt} (val_color {best_val_color:.2f}%)")

    # cleanup hooks
    try:
        h_t.remove(); h_s.remove()
    except:
        pass

    print("CRD training complete. best color-val:", best_val_color)
    return out_student_ckpt

# ---------------------------
# Main pipeline
# ---------------------------
def main():
    start_time = time.time()

    # Step A: finetune teacher with color augmentation
    if FINETUNE_TEACHER:
        print("=== Step A: Color-finetune teacher (may be quick; adjust epochs) ===")
        finetune_teacher_color(TEACHER_CKPT, TEACHER_COLOR_CKPT, epochs=FINETUNE_EPOCHS, lr=FINETUNE_LR)
        color_teacher_ckpt = TEACHER_COLOR_CKPT
    else:
        color_teacher_ckpt = TEACHER_COLOR_CKPT if os.path.exists(TEACHER_COLOR_CKPT) else TEACHER_CKPT

    # Step B: Train CRD student using color-finetuned teacher (teacher frozen)
    if DO_TRAIN_CRD:
        print("=== Step B: Train CRD student using color-finetuned teacher ===")
        train_crd_with_color_teacher(color_teacher_ckpt, STUDENT_CRD_OUT, epochs=CRD_EPOCHS)
    else:
        print("Skipping CRD training (DO_TRAIN_CRD=False)")

    # Step C: Evaluate teacher and student on original & color test sets
    print("=== Step C: Final evaluation summary ===")
    # load test loaders
    _, test_orig, test_color = get_dataloaders(for_teacher=False)

    # teacher eval
    teacher = build_teacher().to(DEVICE)
    teacher.load_state_dict(torch.load(color_teacher_ckpt, map_location=DEVICE))
    teacher.eval()
    t_orig_acc = evaluate_model(teacher, test_orig)
    t_color_acc = evaluate_model(teacher, test_color)
    print(f"Teacher (color-finetuned) - Orig test acc: {t_orig_acc:.2f}% | Color test acc: {t_color_acc:.2f}%")

    # student eval (load best if exists)
    if os.path.exists(STUDENT_CRD_OUT):
        student = build_student().to(DEVICE)
        student.load_state_dict(torch.load(STUDENT_CRD_OUT, map_location=DEVICE))
        student.eval()
        s_orig_acc = evaluate_model(student, test_orig)
        s_color_acc = evaluate_model(student, test_color)
        print(f"Student (CRD from color-teacher) - Orig test acc: {s_orig_acc:.2f}% | Color test acc: {s_color_acc:.2f}%")
    else:
        print("No CRD student checkpoint found at", STUDENT_CRD_OUT)

    elapsed = (time.time() - start_time) / 60.0
    print(f"Total time (minutes): {elapsed:.2f}")
    print("Logs saved to:", LOG_CSV)
    print("Outputs saved to:", TEACHER_COLOR_CKPT, STUDENT_CRD_OUT)

if __name__ == "__main__":
    main()


=== Step A: Color-finetune teacher (may be quick; adjust epochs) ===
Loaded teacher from /content/drive/MyDrive/teacher_vgg16.pth


Finetune Teacher (color) Epoch 1/8: 100%|██████████| 391/391 [00:50<00:00,  7.73it/s, avg_loss=2.23]


Epoch 1: Teacher val orig: 50.95% | val color: 48.57%
  -> Saved color-finetuned teacher to /content/drive/MyDrive/teacher_vgg16_colorfinetune.pth (val_color 48.57%)


Finetune Teacher (color) Epoch 2/8: 100%|██████████| 391/391 [00:54<00:00,  7.18it/s, avg_loss=1.93]


Epoch 2: Teacher val orig: 54.50% | val color: 51.90%
  -> Saved color-finetuned teacher to /content/drive/MyDrive/teacher_vgg16_colorfinetune.pth (val_color 51.90%)


...

Epoch 28: val_orig: 61.39% | val_color: 48.18%


CRD student training epoch 29/30: 100%|██████████| 391/391 [00:56<00:00,  6.90it/s, avg_loss=1.59, ce=0.832, crd=0.754]


Epoch 29: val_orig: 61.66% | val_color: 47.91%


CRD student training epoch 30/30: 100%|██████████| 391/391 [00:56<00:00,  6.88it/s, avg_loss=1.57, ce=0.822, crd=0.752]


Epoch 30: val_orig: 61.84% | val_color: 48.50%
CRD training complete. best color-val: 48.68
=== Step C: Final evaluation summary ===
Teacher (color-finetuned) - Orig test acc: 68.29% | Color test acc: 65.33%
Student (CRD from color-teacher) - Orig test acc: 61.17% | Color test acc: 47.58%
Total time (minutes): 45.88
Logs saved to: /content/drive/MyDrive/crd_color_invariance_log.csv
Outputs saved to: /content/drive/MyDrive/teacher_vgg16_colorfinetune.pth /content/drive/MyDrive/student_vgg11_CRD_colorteacher.pth


##Testing the Efficacy of a Larger Teacher

In [ ]:
# =====================================================
# Part 6: Testing the Efficacy of a Larger Teacher (LM)
# =====================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Using device: {device}")

# ---------------------------
# 1. Load Teacher Models
# ---------------------------
num_classes = 100  # CIFAR-100 (change to 10 if using CIFAR-10)

# Teacher 1: VGG16
teacher_vgg16 = models.vgg16(pretrained=False)
teacher_vgg16.classifier[6] = nn.Linear(4096, num_classes)
teacher_vgg16.load_state_dict(torch.load('/content/drive/MyDrive/teacher_vgg16.pth', map_location=device))
teacher_vgg16.to(device).eval()
print("✅ Loaded Teacher: VGG16")

# Teacher 2: VGG19 (larger teacher)
teacher_vgg19 = models.vgg19(pretrained=True)
teacher_vgg19.classifier[6] = nn.Linear(4096, num_classes)
teacher_vgg19.to(device).eval()
print("✅ Loaded Teacher: VGG19 (pretrained ImageNet weights)")

# ---------------------------
# 2. Student Model (VGG11)
# ---------------------------
def get_student_model():
    model = models.vgg11(pretrained=False)
    model.classifier[6] = nn.Linear(4096, num_classes)
    return model.to(device)

# ---------------------------
# 3. KD Loss (Logit Matching)
# ---------------------------
def kd_loss(student_logits, teacher_logits, temperature=4.0):
    T = temperature
    loss_kd = F.kl_div(
        F.log_softmax(student_logits / T, dim=1),
        F.softmax(teacher_logits / T, dim=1),
        reduction='batchmean'
    ) * (T * T)
    return loss_kd

# ---------------------------
# 4. Training Function
# ---------------------------
def train_kd(student, teacher, trainloader, epochs=3, lr=1e-4, temperature=4.0, alpha=0.7):
    optimizer = torch.optim.Adam(student.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        student.train()
        teacher.eval()

        running_loss, correct, total = 0.0, 0, 0
        pbar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)

        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(images)

            student_logits = student(images)
            loss_ce = criterion(student_logits, labels)
            loss_kd_val = kd_loss(student_logits, teacher_logits, temperature)
            loss = alpha * loss_kd_val + (1 - alpha) * loss_ce

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            correct += (student_logits.argmax(1) == labels).sum().item()
            total += labels.size(0)

            acc = 100.0 * correct / total
            pbar.set_postfix({"Loss": f"{running_loss/total:.4f}", "Acc": f"{acc:.2f}%"})

        print(f"📘 Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/total:.4f} | Acc: {acc:.2f}%")

    print("✅ Training complete.")
    return student

# ---------------------------
# 5. Dataset & Dataloaders
# ---------------------------
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
valset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=2)

# ---------------------------
# 6. Train Student with VGG16
# ---------------------------
print("\n🔵 Training Student (VGG11) with Teacher VGG16")
student_vgg11_v16 = get_student_model()
student_vgg11_v16 = train_kd(student_vgg11_v16, teacher_vgg16, trainloader, epochs=2)
torch.save(student_vgg11_v16.state_dict(), "/content/drive/MyDrive/student_vgg11_from_vgg16.pth")

# ---------------------------
# 7. Train Student with VGG19
# ---------------------------
print("\n🟣 Training Student (VGG11) with Teacher VGG19")
student_vgg11_v19 = get_student_model()
student_vgg11_v19 = train_kd(student_vgg11_v19, teacher_vgg19, trainloader, epochs=2)
torch.save(student_vgg11_v19.state_dict(), "/content/drive/MyDrive/student_vgg11_from_vgg19.pth")

# ---------------------------
# 8. Evaluation Function
# ---------------------------
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total

# ---------------------------
# 9. Compare Models
# ---------------------------
acc_v16 = evaluate(student_vgg11_v16, valloader)
acc_v19 = evaluate(student_vgg11_v19, valloader)

results_df = pd.DataFrame({
    "Teacher Model": ["VGG16", "VGG19"],
    "Teacher Size": ["Medium", "Large"],
    "Student Model": ["VGG11", "VGG11"],
    "Validation Accuracy (%)": [acc_v16, acc_v19]
})

print("\n📊 === Comparison: Teacher Size vs Student Performance ===")
print(results_df.to_string(index=False))


🚀 Using device: cuda
✅ Loaded Teacher: VGG16
✅ Loaded Teacher: VGG19 (pretrained ImageNet weights)

🔵 Training Student (VGG11) with Teacher VGG16


📘 Epoch [1/2] | Loss: 1.3846 | Acc: 6.51%


📘 Epoch [2/2] | Loss: 1.1911 | Acc: 23.31%
✅ Training complete.

🟣 Training Student (VGG11) with Teacher VGG19


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 40.12 MiB is free. Process 2539 has 14.70 GiB memory in use. Of the allocated memory 12.54 GiB is allocated by PyTorch, and 2.02 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()